17 Lands Data starts to update at 00:00 UTC, which is around 8:30pm local. Updating data at 10:30pm is likely a safe time.

In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import seaborn as sns

import WUBRG
import consts
from FormatMetadata import SETS, FORMATS, SET_CONFIG
from JSONHandler import JSONHandler
from FormatMetadata import FormatMetadata

In [2]:
from datetime import date, time, datetime, timedelta

import WUBRG
import consts
from JSONHandler import JSONHandler
from FormatMetadata import FormatMetadata

class RawDataFetcher:    
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self.FORMAT_METADATA = FormatMetadata(SET, FORMAT)
        
        self._META_DICT = dict()
        self._CARD_DICTS = dict()
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    
    @property
    def FORMAT(self):
        """The queue type."""
        return self._FORMAT
    
    
    def get_day_data(self, check_date):
        """
        Gets all of the data for a given day, for the object's set and format.
        If the data does not exist locally, it will be fetched from 17Lands and saved locally.
        :param check_date: The date to get the data for
        :return: A tuple of dictionaries filled with the archetype data and card data
        """
        loader = JSONHandler(self.SET, self.FORMAT, check_date)
        str_date = str(check_date)
        print(f'Getting data for {self.SET} {self.FORMAT}, date: {str_date}')
        card_dict, meta_dict = loader.get_day_data()
        
        self._META_DICT[str_date] = meta_dict
        self._CARD_DICTS[str_date] = {color: card_dict[color] for color in card_dict}
            
        return self._META_DICT[str_date], self._CARD_DICTS[str_date]

    def get_set_data(self):
        """
        Gets all of the data by day for the set and format.
        If any data does not exist locally, it will be fetched from 17Lands and saved locally.
        :return: A tuple of dictionaries filled with the archetype data and card data
        """
        check_date = self.FORMAT_METADATA.START_DATE

        run = True        
        while(run):
            if self.FORMAT_METADATA.is_active(check_date):
                self.get_day_data(check_date)
            check_date += timedelta(days=1)
            run = check_date < date.today() 
    
        self.gen_frames()
        return self._META_DICT, self._CARD_DICTS
    
    def get_summary_data(self):
        """
        Gets the aggregated data for the set and format
        Depending on the age of the data, it will be updated automatically.
        :return: A tuple of dictionaries filled with the archetype data and card data
        """
        loader = JSONHandler(self.SET, self.FORMAT, None) 
        
        # Calculate the last update date
        utc = datetime.utcnow()
        dt = datetime.combine(date(utc.year, utc.month, utc.day), time(2, 0))
        if dt > utc:
            dt -= timedelta(days=1)
        
        # Get the time the file was last updated, and see if it is before the most recent update.
        last_edit = loader.get_last_write_time()
        update = last_edit < dt
        print(f'Getting overall data for {self.SET} {self.FORMAT}')
        card_dict, meta_dict = loader.get_day_data(overwrite=update)            
        return meta_dict, card_dict

In [9]:
ACTIVE_SET = 'NEO'
ACTIVE_FORMAT = 'PremierDraft'
loader = JSONHandler(ACTIVE_SET, ACTIVE_FORMAT)
last_edit = loader.get_last_write_time()
last_edit

datetime.datetime(2022, 2, 16, 23, 5, 17, 631923)

In [5]:
utc = datetime.utcnow()
utc

datetime.datetime(2022, 2, 17, 0, 3, 38, 505782)

In [6]:
dt = datetime.combine(date(utc.year, utc.month, utc.day), time(2, 0))
dt

datetime.datetime(2022, 2, 17, 2, 0)

In [7]:
if dt > utc:
    dt -= timedelta(days=1)
dt

datetime.datetime(2022, 2, 16, 2, 0)

In [8]:
update = last_edit < dt
update

True

In [ ]:
ACTIVE_SET = 'NEO'
ACTIVE_FORMAT = 'PremierDraft'

fetcher = RawDataFetcher(ACTIVE_SET, ACTIVE_FORMAT)

In [ ]:
fetcher.get_summary_data()

In [ ]:
from datetime import date, time, datetime, timedelta
import pandas as pd

import WUBRG
import consts
from JSONHandler import JSONHandler

class RawDataHandler:    
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self.FORMAT_METADATA = FormatMetadata(SET, FORMAT)
        
        self._SUMMARY_DICT = dict()
        self._ARCHTYPE_DICT = dict()
        self._CARD_DICTS = dict()
        
        self.SUMMARY_FRAME = None
        self.ARCHTYPE_FRAME = None
        self.CARD_FRAME = None
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    
    @property
    def FORMAT(self):
        """The queue type."""
        return self._FORMAT
    
    
    def panadafy_card_dict(self, card_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(card_dict)
        frame = frame.rename(columns=consts.STAT_NAMES)

        # If there's no data, make a blank frame and return it.
        if len(card_dict) == 0:
            return frame

        frame = frame.set_index('Name')
        
        for col in ["GP WR", "OH WR", "GD WR", "GIH WR", "GND WR", "IWD"]:
            frame[col] = frame[col] * 100

        frame = frame.drop(['sideboard_game_count', 'sideboard_win_rate', 'url', 'url_back'], axis=1)
        frame['Rarity'] = frame['Rarity'].map(consts.RARITY_ALIASES)
        frame = frame.round(3)
        return frame
    
    def panadafy_meta_dict(self, meta_dict: dict) -> pd.DataFrame:
        """
        Turns a dictionary into a DataFrame, with some data cleaning applied.
        :param card_dict: The dictionary containing card data for a colour group
        :return: A DataFrame filled with the cleaned card data
        """
        frame = pd.DataFrame.from_dict(meta_dict)
        frame = frame.rename(columns=consts.META_COLS)

        # If there's no data, make a blank frame and return it.
        if len(meta_dict) == 0:
            return frame, frame.copy()
        
        frame['Name'] = frame['Color Name']
        frame = frame.set_index('Name')
        frame['Win %'] = round((frame['Wins'] / frame['Games']) * 100, 2)
        frame['Splash'] = frame['Color Name'].str.contains("Splash", case=False)
        frame['Colors'] = frame['Color Name'].map(lambda x: x.replace(' + Splash', ''))
        frame = frame[['Colors', 'Splash', 'Wins', 'Games', 'Win %', 'is_summary']]
        
        summary_frame = frame[frame['is_summary'] == True].copy()
        summary_frame = summary_frame.drop(['is_summary'], axis=1)
        summary_frame['Colors'] = summary_frame['Colors'].map(WUBRG.COLOR_COUNT_MAP)

        archetype_frame = frame[frame['is_summary'] == False].copy()
        archetype_frame = archetype_frame.drop(['is_summary'], axis=1)
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x[0: (x.find('(') if x.find('(') != -1 else len(x))].strip())
        archetype_frame['Colors'] = archetype_frame['Colors'].map(lambda x: x.replace('Mono-', ''))
        archetype_frame['Colors'] = archetype_frame['Colors'].map(WUBRG.COLOR_ALIASES)
        archetype_frame['Name'] = archetype_frame['Colors']
        archetype_frame = archetype_frame.set_index('Name')

        return summary_frame, archetype_frame
    
    def gen_frames(self):
        self.SUMMARY_FRAME = pd.concat(self._SUMMARY_DICT, names=["Date", "Name"])
        self.ARCHTYPE_FRAME = pd.concat(self._ARCHTYPE_DICT, names=["Date", "Name"])
        temp_dict = dict()
        for date in self._CARD_DICTS:
            temp_dict[date] = pd.concat(self._CARD_DICTS[date], names=["Deck Colors", "Name"])
        self.CARD_FRAME = pd.concat(temp_dict, names=["Date", "Deck Colors", "Name"])
        
    def get_day_data(self, check_date):
        loader = JSONHandler(self.SET, self.FORMAT, check_date)
        str_date = str(check_date)
        print(f'Getting data for {self.SET} {self.FORMAT}, date: {str_date}')
        card_dict, meta_dict = loader.get_day_data()
        
        self._SUMMARY_DICT[str_date], self._ARCHTYPE_DICT[str_date] = self.panadafy_meta_dict(meta_dict)
        self._CARD_DICTS[str_date] = dict()
        for color in card_dict:
            self._CARD_DICTS[str_date][color] = self.panadafy_card_dict(card_dict[color])
            
        return self._SUMMARY_DICT[str_date], self._ARCHTYPE_DICT[str_date], self._CARD_DICTS[str_date]

    def get_set_data(self):
        check_date = self.FORMAT_METADATA.START_DATE

        run = True        
        while(run):
            if self.FORMAT_METADATA.is_active(check_date):
                self.get_day_data(check_date)
            check_date += timedelta(days=1)
            run = check_date < date.today() 
    
        self.gen_frames()
        return self._SUMMARY_DICT, self._ARCHTYPE_DICT, self._CARD_DICTS
    
    def get_summary_data(self):
        loader = JSONHandler(self.SET, self.FORMAT, None)
        print(f'Getting overall data for {self.SET} {self.FORMAT}')
        # TODO: Determine when to update based on a time-stamp.
        # NOTE: This might not be necessary, as there isn't a need to automatically update. It may be better
        # to call this ad-hoc.
        card_dict, meta_dict = loader.get_day_data(overwrite=True)
        
        summary, archetype = self.panadafy_meta_dict(meta_dict)
        cards = dict()
        for color in card_dict:
            cards[color] = self.panadafy_card_dict(card_dict[color])
            
        return summary, archetype, cards

In [ ]:
class RawDataWrapper:   
    def __init__(self, SET, FORMAT):
        self._SET = SET
        self._FORMAT = FORMAT
        self._DATA = RawDataHandler(SET, FORMAT)
    
    
    @property
    def SET(self):
        """The draft set."""
        return self._SET
    
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._FORMAT
    
    
    @property
    def DATA(self):
        """The data about the set and format."""
        return self._DATA
    

    def get_set_data(self):
        self.DATA.get_set_data()
        
    def get_summary_data(self):
        return self.DATA.get_summary_data()
    
    def summary_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.SUMMARY_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
     
    def archetype_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.ARCHTYPE_FRAME.loc(axis=0)[pd.IndexSlice[date, name]]
    
    def card_frame(self, date=slice(None, None, None), colors=slice(None, None, None), name=slice(None, None, None)):
        return self.DATA.CARD_FRAME.loc(axis=0)[pd.IndexSlice[date, colors, name]]

In [ ]:
class DataManager:
    def __init__(self):        
        self.DATA = {s: {f: RawDataWrapper(s, f) for f in FORMATS} for s in SETS}
        
        self._SET = SETS[0]
        self._FORMAT = FORMATS[0]
        self._ACTIVE_DATA = self.DATA[self._SET][self._FORMAT]
        
    @property
    def SET(self):
        """The draft set."""
        return self._SET

    @SET.setter
    def SET(self, value):
        if value in SETS:
            self._SET = value
            self._ACTIVE_DATA = self.DATA[self._SET][self._FORMAT]
        else:
            raise ValueError(f"Inavlid set: '{value}'")
        
    @property
    def FORMAT(self):
        """The draft format."""
        return self._FORMAT

    @FORMAT.setter
    def FORMAT(self, value):
        if value in FORMATS:
            self._FORMAT = value
            self._ACTIVE_DATA = self.DATA[self.SET][self.FORMAT]
        else:
            raise ValueError(f"Inavlid format: '{value}'")
            
    
    @property
    def ACTIVE_DATA(self):
        """The data for the given set and format."""
        return self._ACTIVE_DATA
    
    def get_set_data(self):
        self.ACTIVE_DATA.get_set_data()
    
    def summary_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.summary_frame(date, name)
     
    def archetype_frame(self, date=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.archetype_frame(date, name)
    
    def card_frame(self, date=slice(None, None, None), colors=slice(None, None, None), name=slice(None, None, None)):
        return self.ACTIVE_DATA.card_frame(date, colors, name)

In [ ]:
manager = DataManager()
manager.SET = 'NEO'
manager.FORMAT = 'PremierDraft'

In [ ]:
manager.get_set_data()

In [ ]:
color_pairs = [''] + [WUBRG.COLOR_ALIASES_SUPPORT['Guilds'][key] for key in WUBRG.COLOR_ALIASES_SUPPORT['Guilds']]
color_pairs

In [ ]:
card_array = {col: manager.card_frame(name='Eiganjo Exemplar', colors=col)[['GIH WR', 'ALSA', '# GP', 'IWD']] for col in color_pairs}
for key in card_array:
    array = card_array[key]
    new_idxs = [tup[0][5:] for tup in array.index]
    array.index = new_idxs
card_array

In [ ]:
#frame = pd.concat(card_array)
frame = card_array['']
frame.plot(subplots=True, layout=(2,2), figsize=(8,8))

In [ ]:
manager.card_frame(name='Eiganjo Exemplar', colors='WG')

In [ ]:
manager.card_frame(name='Bearer of Memory', colors='WG')

In [ ]:
day_frame = manager.archetype_frame(date='2022-02-14')
win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False]
day_frame['Games'].sum()
day_frame

In [ ]:
def get_winrate(day=None):
    if day:
        return manager.summary_frame().loc[(day, 'All Decks')]['Win %']
    else:
        # TODO: Get the overall winrate 
        return None

In [ ]:
def get_archetype_winrate_history(color_filter=None, roll=1):
    color_pairs = [''] + [WUBRG.COLOR_ALIASES_SUPPORT['Guilds'][key] for key in WUBRG.COLOR_ALIASES_SUPPORT['Guilds']]
    d = dict()
    for col in color_pairs:
        if col == '':
            d['AVG'] = graph_archtype_winrates_frame('UR')['Avg. Win%']
        else:
            d[col] = graph_archtype_winrates_frame(col)['Win %']
    test_frame = pd.DataFrame.from_dict(d)
    if color_filter:
        col_filt = ['AVG'] + [col for col in color_pairs if color_filter in col]
        test_frame = test_frame[col_filt]
    rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
    return rolling

In [ ]:
test_frame = get_archetype_winrate_history('R', 7)
colors =[(0.85,0.85,0.2), (0,0,1), (0,0,0), (1,0,0), (0,0.7,0)]
test_frame.plot(figsize=(20, 10), color=colors)

# Old Tests

In [ ]:
pd.options.display.float_format = None

In [ ]:
frame = today[2][''].copy()
avg_wr = today[0].loc['All Decks']['Win %']
wu_avg_wr = today[1][today[1]['Colors'] == 'WU'][today[1]['Splash'] == False].iloc[0]['Win %']
#frame['Δ WR'] = None
frame = frame.drop(['# Seen', '# Picked', '# OH', '# GD', '# GIH', '# GND'], axis=1)
frame['(x-μ) GP WR'] = frame['GP WR'] - avg_wr
frame['(x-μ) GIH WR'] = frame['GIH WR'] - avg_wr

frame.sort_values(by=['(x-μ) GP WR'])
frame[frame['(x-μ) GP WR'] < frame['(x-μ) GIH WR']].sort_values(by=['(x-μ) GIH WR'])
frame['Spikiness'] =  frame['GIH WR'] - frame['GP WR']
frame

# Finished Functions

## Card Pick Order Stats

In [ ]:
def graph_pick_stats(card_name, roll=1):
    taken_data = manager.card_frame(name=card_name, colors='')[['ALSA', 'ATA']]
    taken_data.index = [tup[0][5:] for tup in taken_data.index]
    rolling = taken_data.rolling(window=roll, min_periods=1, center=True).mean()
    
    mx = min(max(taken_data.max()) + 0.25, 15) 
    mn = max(min(taken_data.min()) - 0.25, 1)
    tit_str = f"{manager.SET} - {manager.FORMAT}\n{card_name}"
    
    rolling.plot(ylim=(mx, mn), grid=True, title=tit_str)
    
    return taken_data

In [ ]:
graph_pick_stats('Voldaren Epicure', 1)

In [ ]:
graph_pick_stats('Imperial Oath', 1)

In [ ]:
graph_pick_stats('Behold the Unspeakable', 3)

In [ ]:
to_graph = ['Imperial Oath', 'Behold the Unspeakable', 'Virus Beetle', 'Network Disruptor']
for card in to_graph:
    graph_pick_stats(card, 1)

## Archetype Winrate History

In [ ]:
def graph_archtype_winrates_frame(colors, roll=1):
    colors = WUBRG.get_color_identity(colors)
    win_rate_frame = manager.archetype_frame(name=colors)
    win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
    #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
    win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
    rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
    rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
    rolling['Avg. Win%'] = [get_winrate(idx) for idx in win_rate_frame.index]
    rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
    return rolling

In [ ]:
graph_archtype_winrates_frame('RW', 1)

In [ ]:
graph_archtype_winrates_frame('RW', 3)